# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [214]:
data["customerName"] = data["FirstName"]+data["LastName"]
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity,customerName
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16,RosaAndersen
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20,MyronMurray
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11,SusanStevenson
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6,TriciaVincent
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20,ScottBurch


In [59]:
len(data["customerName"].unique())

999

In [68]:
"""I have decided to do the lab using customerName in stead of CustomerID, loosing one record"""

'I have decided to do the lab using customerName in stead of CustomerID, loosing one record'

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [213]:
data_1 = data.groupby(["CustomerID","ProductName","customerName"],as_index=False).agg(alvaro_sum=("Quantity","sum"))
data_1.head()

,CustomerID,ProductName,customerName,alvaro_sum
0,33,Apricots - Dried,LindsaySantana,1
1,33,Assorted Desserts,LindsaySantana,1
2,33,Bandage - Flexible Neon,LindsaySantana,1
3,33,"Bar Mix - Pina Colada, 355 Ml",LindsaySantana,1
4,33,"Beans - Kidney, Canned",LindsaySantana,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [212]:
data_2 = data_1.pivot_table(index=["ProductName"], columns=["customerName"], values=["alvaro_sum"], aggfunc="sum",fill_value=0)
data_2 = data_2["alvaro_sum"]
data_2_transposed = data_2.T
data_2_transposed.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
customerName,,,,,,,,,,,,,,,,,,,,,
AaronEdwards,0,8,0,8,0,0,0,0,0,0,...,0,0,8,8,0,0,0,0,0,8
AaronWashington,2,2,0,0,0,0,0,0,0,0,...,0,2,0,2,0,0,0,0,0,0
AbelHoffman,0,0,0,0,0,0,0,0,0,0,...,21,0,0,21,0,0,0,0,0,0
AbigailCharles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AdamBarr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,9,0,0,9


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [211]:
#pdist()
#squareform()
#1/(1+ dataFrame)
#pd.DataFrame()
pdist_calculation_squared_norm = pd.DataFrame(1/(1+ squareform(pdist(X=data_2_transposed, metric='euclidean'))),
                                             index=data_2_transposed.index,
                                             columns=data_2_transposed.index)
pdist_calculation_squared_norm.head()

customerName,AaronEdwards,AaronWashington,AbelHoffman,AbigailCharles,AdamBarr,AdrianaMoran,AimeeWare,AishaMunoz,AlanaCarroll,AlbertAguirre,...,WarrenRobles,WendyWilkins,WesleyGardner,WesleyHendrix,WesleyOlsen,WilliamWelch,XavierOwen,YeseniaBender,YvonneHale,YvonneMerritt
customerName,,,,,,,,,,,,,,,,,,,,,
AaronEdwards,1.000000,0.015437,0.005453,0.004254,0.009418,0.008547,0.006187,0.015289,0.012533,0.004832,...,0.012076,0.009065,0.005103,0.015326,0.012189,0.012091,0.009354,0.013305,0.006543,0.008768
AaronWashington,0.015437,1.000000,0.005558,0.004375,0.010905,0.009687,0.006398,0.040825,0.018406,0.004842,...,0.017077,0.010180,0.005323,0.042807,0.017555,0.017141,0.010316,0.018145,0.006791,0.009954
AbelHoffman,0.005453,0.005558,1.000000,0.003588,0.005195,0.004964,0.004535,0.005584,0.005491,0.004154,...,0.005386,0.005139,0.004068,0.005559,0.005441,0.005583,0.005278,0.005536,0.004570,0.005070
AbigailCharles,0.004254,0.004375,0.003588,1.000000,0.004189,0.004104,0.003778,0.004361,0.004314,0.003461,...,0.004324,0.004204,0.003473,0.004340,0.004406,0.004306,0.004174,0.004281,0.003863,0.004108
AdamBarr,0.009418,0.010905,0.005195,0.004189,1.000000,0.007699,0.005938,0.011058,0.009924,0.004619,...,0.009604,0.008646,0.005114,0.010948,0.010069,0.009684,0.008264,0.010280,0.006133,0.007773


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [81]:
#CUSTOMER CHOOSED = AaronEdwards
data_4_simil = pdist_calculation_squared_norm["AaronEdwards"].sort_values(ascending=False)[1:]
data_4_simil_top5 = data_4_simil.head()
data_4_simil_top5_dic_items = dict(data_4_simil_top5).items()
data_4_simil_top5_dic_items

dict_items([('TraceyWalters', 0.01592200800247441), ('HopeIrwin', 0.01586083871913595), ('LeoCross', 0.015832534577188768), ('TysonShepherd', 0.01575051209013114), ('VirgilSmall', 0.015738615973150457)])

In [101]:
data_4_simil_top5_list = data_4_simil_top5.index.tolist()
data_4_simil_top5_list

['TraceyWalters', 'HopeIrwin', 'LeoCross', 'TysonShepherd', 'VirgilSmall']

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [210]:
data_5 = data_1[data_1["customerName"].isin(data_4_simil_top5_list)]
data_5.head()

,CustomerID,ProductName,customerName,alvaro_sum
131,264,Apricots - Halves,VirgilSmall,1
132,264,Apricots Fresh,VirgilSmall,1
133,264,Bacardi Breezer - Tropical,VirgilSmall,1
134,264,Bagel - Plain,VirgilSmall,1
135,264,Banana - Leaves,VirgilSmall,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [209]:
data_6 = data_5.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
data_6.head()

,ProductName,alvaro_sum
0,Tomatoes Tear Drop,10
1,Appetizer - Sausage Rolls,9
2,Bacardi Breezer - Tropical,9
3,"Pepsi - Diet, 355 Ml",8
4,Berry Brulee,8


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [129]:
data_AaronEdwards = data_1[data_1["customerName"].isin(["AaronEdwards"])]
data_AaronEdwards_products = data_AaronEdwards["ProductName"].unique().tolist()
#data_AaronEdwards_products
data_7 = data_6.drop(data_6[data_6["ProductName"].isin(data_AaronEdwards_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
data_7.head()

,ProductName,alvaro_sum
0,"Pepsi - Diet, 355 Ml",8
1,Berry Brulee,8
2,Table Cloth 120 Round White,7
3,Mangoes,7
4,Chips Potato Salt Vinegar 43g,7


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [166]:
step_8_dic_from_list = {}
step_8_dic_from_dict = {}
step_8_dic_from_df = {}

step_8_customer_name_list = data["customerName"].unique().tolist()

In [197]:
#RECOMENDATIONS LIST
for name in step_8_customer_name_list:
    data_8_step4_simil_list = pdist_calculation_squared_norm[name].sort_values(ascending=False)[1:6].index.tolist()
    data_8_step5_similar_records = data_1[data_1["customerName"].isin(data_8_step4_simil_list)]
    data_8_step6_aggregation = data_8_step5_similar_records.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
    data_8_step7_user_product_consumed_df = data_1[data_1["customerName"].isin([name])]
    data_8_step7_user_product_consumed_products = data_8_step7_user_product_consumed_df["ProductName"].unique().tolist()
    data_8_step7_top5 = data_8_step6_aggregation.drop(data_8_step6_aggregation[data_8_step6_aggregation["ProductName"].isin(data_8_step7_user_product_consumed_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True).head()
    
    step_8_dic_from_list[name] = data_8_step7_top5["ProductName"].tolist()

In [201]:
step_8_dic_from_list["RosaAndersen"]

['Jagermeister',
 'Chicken - Soup Base',
 'Lime Cordial - Roses',
 'Macaroons - Two Bite Choc',
 'Flavouring - Orange']

In [202]:
#RECOMENDATIONS DICT
for name in step_8_customer_name_list:
    data_8_step4_simil_list = pdist_calculation_squared_norm[name].sort_values(ascending=False)[1:6].index.tolist()
    data_8_step5_similar_records = data_1[data_1["customerName"].isin(data_8_step4_simil_list)]
    data_8_step6_aggregation = data_8_step5_similar_records.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
    data_8_step7_user_product_consumed_df = data_1[data_1["customerName"].isin([name])]
    data_8_step7_user_product_consumed_products = data_8_step7_user_product_consumed_df["ProductName"].unique().tolist()
    data_8_step7_top5 = data_8_step6_aggregation.drop(data_8_step6_aggregation[data_8_step6_aggregation["ProductName"].isin(data_8_step7_user_product_consumed_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True).head()
    
    step_8_dic_from_dict[name] = data_8_step7_top5.to_dict()

In [203]:
step_8_dic_from_dict["RosaAndersen"]

{'ProductName': {0: 'Jagermeister',
  1: 'Chicken - Soup Base',
  2: 'Lime Cordial - Roses',
  3: 'Macaroons - Two Bite Choc',
  4: 'Flavouring - Orange'},
 'alvaro_sum': {0: 12, 1: 12, 2: 10, 3: 9, 4: 8}}

In [204]:
#RECOMENDATIONS DF
for name in step_8_customer_name_list:
    data_8_step4_simil_list = pdist_calculation_squared_norm[name].sort_values(ascending=False)[1:6].index.tolist()
    data_8_step5_similar_records = data_1[data_1["customerName"].isin(data_8_step4_simil_list)]
    data_8_step6_aggregation = data_8_step5_similar_records.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
    data_8_step7_user_product_consumed_df = data_1[data_1["customerName"].isin([name])]
    data_8_step7_user_product_consumed_products = data_8_step7_user_product_consumed_df["ProductName"].unique().tolist()
    data_8_step7_top5 = data_8_step6_aggregation.drop(data_8_step6_aggregation[data_8_step6_aggregation["ProductName"].isin(data_8_step7_user_product_consumed_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True).head()
    
    step_8_dic_from_df[name] = data_8_step7_top5

In [205]:
step_8_dic_from_df["RosaAndersen"]

,ProductName,alvaro_sum
0,Jagermeister,12
1,Chicken - Soup Base,12
2,Lime Cordial - Roses,10
3,Macaroons - Two Bite Choc,9
4,Flavouring - Orange,8


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [206]:
step_9 = pd.DataFrame({"customerName":step_8_dic_from_list.keys(),"recommendatedProducts":step_8_dic_from_list.values()})
step_9.head()

,customerName,recommendatedProducts
0,RosaAndersen,"[Jagermeister, Chicken - Soup Base, Lime Cordi..."
1,MyronMurray,"[Wine - Valpolicella Masi, Tahini Paste, Tuna ..."
2,SusanStevenson,"[Bread - Italian Corn Meal Poly, Puree - Mocha..."
3,TriciaVincent,"[Squid - Tubes / Tenticles 10/20, Wanton Wrap,..."
4,ScottBurch,"[Vanilla Beans, Lettuce - Frisee, Yogurt - Blu..."


In [207]:
step_9_2 = pd.DataFrame({"customerName":step_8_dic_from_list.keys(),"recommendatedProduct1":[i[0] for i in list(step_8_dic_from_list.values())],"recommendatedProduct2":[i[1] for i in list(step_8_dic_from_list.values())],"recommendatedProduct3":[i[2] for i in list(step_8_dic_from_list.values())],"recommendatedProduct4":[i[3] for i in list(step_8_dic_from_list.values())],"recommendatedProduct5":[i[4] for i in list(step_8_dic_from_list.values())]})
step_9_2.head()

,customerName,recommendatedProduct1,recommendatedProduct2,recommendatedProduct3,recommendatedProduct4,recommendatedProduct5
0,RosaAndersen,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,MyronMurray,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,SusanStevenson,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,TriciaVincent,Squid - Tubes / Tenticles 10/20,Wanton Wrap,Puree - Mocha,Pernod,Sprouts - Baby Pea Tendrils
4,ScottBurch,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [208]:
#pdist()
#squareform()
#1/(1+ dataFrame)
#pd.DataFrame()
step_10_pdist_calculation_squared_norm = pd.DataFrame(1/(1+ squareform(pdist(X=data_2_transposed, metric='cosine'))),
                                             index=data_2_transposed.index,
                                             columns=data_2_transposed.index)
step_10_pdist_calculation_squared_norm.head()

customerName,AaronEdwards,AaronWashington,AbelHoffman,AbigailCharles,AdamBarr,AdrianaMoran,AimeeWare,AishaMunoz,AlanaCarroll,AlbertAguirre,...,WarrenRobles,WendyWilkins,WesleyGardner,WesleyHendrix,WesleyOlsen,WilliamWelch,XavierOwen,YeseniaBender,YvonneHale,YvonneMerritt
customerName,,,,,,,,,,,,,,,,,,,,,
AaronEdwards,1.000000,0.541353,0.535238,0.520344,0.529490,0.522953,0.534732,0.527799,0.529606,0.549625,...,0.527799,0.534012,0.519583,0.526378,0.527818,0.517542,0.546274,0.558957,0.538891,0.529931
AaronWashington,0.541353,1.000000,0.520299,0.551069,0.529105,0.527436,0.523508,0.520903,0.551371,0.543630,...,0.551653,0.523031,0.545704,0.523310,0.558915,0.512349,0.524294,0.533336,0.530636,0.542826
AbelHoffman,0.535238,0.520299,1.000000,0.525657,0.530754,0.517730,0.540150,0.533849,0.531285,0.568031,...,0.518256,0.531709,0.531245,0.520346,0.525955,0.548766,0.546697,0.539040,0.532377,0.527919
AbigailCharles,0.520344,0.551069,0.525657,1.000000,0.530097,0.525657,0.528638,0.540720,0.527679,0.537075,...,0.533479,0.538114,0.519035,0.521809,0.554882,0.526808,0.531792,0.518237,0.532024,0.524719
AdamBarr,0.529490,0.529105,0.530754,0.530097,1.000000,0.530754,0.543498,0.549088,0.532417,0.535080,...,0.525034,0.574006,0.549186,0.531655,0.548676,0.522443,0.548093,0.552297,0.536826,0.530517


In [194]:
step_10_data_4_simil = step_10_pdist_calculation_squared_norm["AaronEdwards"].sort_values(ascending=False)[1:]
step_10_data_4_simil_top5 = step_10_data_4_simil.head()
step_10_data_4_simil_top5_dic_items = dict(step_10_data_4_simil_top5).items()

step_10_data_4_simil_top5_list = step_10_data_4_simil_top5.index.tolist()

step_10_data_5 = data_1[data_1["customerName"].isin(data_4_simil_top5_list)]

step_10_data_6 = data_5.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)

step_10_data_AaronEdwards = data_1[data_1["customerName"].isin(["AaronEdwards"])]
step_10_data_AaronEdwards_products = step_10_data_AaronEdwards["ProductName"].unique().tolist()
step_10_data_7 = step_10_data_6.drop(step_10_data_6[step_10_data_6["ProductName"].isin(step_10_data_AaronEdwards_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
step_10_data_7.head()

,ProductName,alvaro_sum
0,"Pepsi - Diet, 355 Ml",8
1,Berry Brulee,8
2,Table Cloth 120 Round White,7
3,Mangoes,7
4,Chips Potato Salt Vinegar 43g,7


In [195]:
"""The result in this case is the same"""

'The result in this case is the same'